In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, roc_auc_score,accuracy_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from eli5.sklearn import PermutationImportance
from sklearn.model_selection import cross_val_score
import eli5
import ast
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.feature_selection import f_classif
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.metrics import precision_recall_curve,auc
import seaborn as sns
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from pretty_confusion_matrix import pp_matrix

In [3]:
plasma_rf=pd.read_csv("R/GEO_count/pan_train2.csv")

In [4]:
plasma_rf=plasma_rf.drop(columns=['Unnamed: 0'])

In [5]:
X_train = plasma_rf.drop(columns=['group'])
y_train = plasma_rf['group']

In [20]:
test=pd.read_csv("R/GEO_count/pan_test2.csv")
test=pd.read_csv("R/GEO_count/tissue_test2.csv")
test=pd.read_csv("R/GEO_count/plasma_test2.csv")
test=pd.read_csv("R/GEO_count/exosome_test2.csv")
test=pd.read_csv("R/GEO_count/exosome_test2_CH.csv")
test=pd.read_csv("R/GEO_count/exosome_test2_BC.csv")
#independent cohort
test=pd.read_csv("R/GEO_count/GSE62182_rf.csv")
test=pd.read_csv("R/GEO_count/GSE83527_rf.csv")

In [21]:
test=test.drop(columns=['Unnamed: 0'])

In [22]:
X_test=test.drop(columns=['group'])
y_test=test['group']

In [ ]:
rf_classifier = RandomForestClassifier(random_state=42)

In [ ]:
rf_classifier.fit(X_train, y_train)

In [ ]:
feature_importances = rf_classifier.feature_importances_
feature_names = X_train.columns
sorted_importances, sorted_features = zip(*sorted(zip(feature_importances, feature_names), reverse=True))
for feature, importance in zip(sorted_features, sorted_importances):
    print(f"Feature: {feature}, Importance: {importance}")

In [ ]:
df = pd.DataFrame({'Features': sorted_features, 'Importances': sorted_importances})

# Sort the DataFrame by Importances
df = df.sort_values('Importances', ascending=False)

# Bar Plot
plt.figure(figsize=(12, 6))
sns.barplot(x='Importances', y='Features', data=df,palette='YlGnBu')
plt.xlabel('Importance')
plt.ylabel('Features')
plt.title('Feature Importances')
plt.show()

In [ ]:
param_grid = {
    'n_estimators': [50, 100, 200,300],
    'max_features': ['sqrt', 2, 3, 4, 5],
    'max_depth': [2, 3, 4, 5, None],
    'min_samples_split': [2, 3, 4],
   'min_samples_leaf': [1, 2, 3],
    'bootstrap': [True, False]
}

In [ ]:
grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid,
                           cv=5, n_jobs=50, verbose=2)

In [ ]:
grid_search.fit(X_train, y_train)

In [ ]:
best_params = grid_search.best_params_
best_params = {'bootstrap': True, 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 50}
print("Best parameters found: ", best_params)

In [ ]:
best_rf_classifier = RandomForestClassifier(**best_params,random_state=42)
best_rf_classifier.fit(X_train, y_train)

In [ ]:
accuracy_train = best_rf_classifier.score(X_train, y_train)
print("train set accuracy: ", accuracy_train)

average_cv_accuracy = grid_search.best_score_
print("Average CV accuracy: ", average_cv_accuracy)

In [ ]:
accuracy_test = best_rf_classifier.score(X_test, y_test)

print("Test set accuracy: ", accuracy_test)

In [ ]:
y_pred_proba_train = best_rf_classifier.predict_proba(X_train)[:, 1]
fpr_train, tpr_train, thresholds_train = roc_curve(y_train, y_pred_proba_train, pos_label='tumor')
auc_value_train = roc_auc_score(y_train, y_pred_proba_train)
print("AUC_train:",auc_value_train)
youden_J_train = tpr_train - fpr_train

# Locate the index of the largest J statistic
ix = np.argmax(youden_J_train)
optimal_threshold_train = thresholds_train[ix]

print('Best Threshold=%f, Youden J=%.3f' % (optimal_threshold_train, youden_J_train[ix]))

In [ ]:
y_pred_proba_test = best_rf_classifier.predict_proba(X_test)[:, 1]
fpr_test, tpr_test, thresholds_test = roc_curve(y_test, y_pred_proba_test, pos_label='tumor')
auc_value_test = roc_auc_score(y_test, y_pred_proba_test)
print("AUC_test:",auc_value_test)
youden_J_test = tpr_test - fpr_test

# Locate the index of the largest J statistic
ix_test = np.argmax(youden_J_test)
optimal_threshold_test = thresholds_test[ix_test]

print('Best Threshold=%f, Youden J=%.3f' % (optimal_threshold_test, youden_J_test[ix_test]))

In [ ]:
#threshold at 0.5
y_pred_train=best_rf_classifier.predict(X_train)
f1_train=f1_score(y_train,y_pred_train,pos_label='tumor')
precision_train_score = precision_score(y_train, y_pred_train, pos_label='tumor')
recall_train_score = recall_score(y_train, y_pred_train, pos_label='tumor')
print(f"F1-Score_train: {f1_train}")
print(f"Precision_train: {precision_train_score}")
print(f"Recall_train: {recall_train_score}")

y_pred_test=best_rf_classifier.predict(X_test)
f1_test=f1_score(y_test,y_pred_test,pos_label='tumor')
precision_test_score = precision_score(y_test, y_pred_test, pos_label='tumor')
recall_test_score = recall_score(y_test, y_pred_test, pos_label='tumor')
print(f"F1-Score_test: {f1_test}")
print(f"Precision_test: {precision_test_score}")
print(f"Recall_test: {recall_test_score}")

In [ ]:
precision_train, recall_train, thresholds_train = precision_recall_curve(y_train, y_pred_proba_train,pos_label='tumor')
f1_scores_train = 2 * (precision_train * recall_train) / (precision_train + recall_train)
# Handle division by zero in case precision and recall are both zero
f1_scores_train = np.nan_to_num(f1_scores_train)

optimal_idx_train = np.argmax(f1_scores_train)
optimal_threshold_train = thresholds_test[optimal_idx_train]
best_f1_score_train = f1_scores_train[optimal_idx_train] 
print("Optimal threshold:", optimal_threshold_train)
print("best f score:", best_f1_score_train)


y_pred_optimal_threshold_train = np.where(y_pred_proba_train >= optimal_threshold_train, "tumor", "normal")

tn_train, fp_train, fn_train, tp_train = confusion_matrix(y_train, y_pred_optimal_threshold_train).ravel()

ppv_train = tp_train / (tp_train + fp_train) if (tp_train + fp_train) != 0 else 0

# Negative Predictive Value
npv_train = tn_train / (tn_train + fn_train) if (tn_train + fn_train) != 0 else 0

# Sensitivity (Recall)
sensitivity_train = tp_train / (tp_train + fn_train) if (tp_train + fn_train) != 0 else 0

# Specificity
specificity_train = tn_train / (tn_train + fp_train) if (tn_train + fp_train) != 0 else 0

#AUPRC
auprc_train = auc(recall_train, precision_train)
print("auprc_train:", auprc_train)

print(f"PPV/Precision: {ppv_train}")
print(f"NPV: {npv_train}")
print(f"Sensitivity/Recall: {sensitivity_train}")
print(f"Specificity: {specificity_train}")

In [ ]:
precision_test, recall_test, thresholds_test = precision_recall_curve(y_test, y_pred_proba_test,pos_label='tumor')
f1_scores_test = 2 * (precision_test * recall_test) / (precision_test + recall_test)
# Handle division by zero in case precision and recall are both zero
f1_scores_test = np.nan_to_num(f1_scores_test)

optimal_idx_test = np.argmax(f1_scores_test)
optimal_threshold_test = thresholds_test[optimal_idx_test]
best_f1_score_test = f1_scores_test[optimal_idx_test] 
print("Optimal threshold:", optimal_threshold_test)
print("best f score:", best_f1_score_test)


y_pred_optimal_threshold_test = np.where(y_pred_proba_test >= optimal_threshold_test, "tumor", "normal")

tn_test, fp_test, fn_test, tp_test = confusion_matrix(y_test, y_pred_optimal_threshold_test).ravel()

ppv_test = tp_test / (tp_test + fp_test) if (tp_test + fp_test) != 0 else 0

# Negative Predictive Value
npv_test = tn_test / (tn_test + fn_test) if (tn_test + fn_test) != 0 else 0

# Sensitivity (Recall)
sensitivity_test = tp_test / (tp_test + fn_test) if (tp_test + fn_test) != 0 else 0

# Specificity
specificity_test = tn_test / (tn_test + fp_test) if (tn_test + fp_test) != 0 else 0


#AUPRC
auprc_test = auc(recall_test, precision_test)
print("auprc_test:", auprc_test)

print(f"PPV/Precision: {ppv_test}")
print(f"NPV: {npv_test}")
print(f"Sensitivity/Recall: {sensitivity_test}")
print(f"Specificity: {specificity_test

In [ ]:
#AUC curve
# Assuming fpr_val, tpr_val, and auc_val contain the FPR, TPR, and AUC for the validation set

plt.figure(figsize=(5, 5))

# plot ROC curve for the train set
plt.plot(fpr_train, tpr_train, label=f'Train Set AUC = {auc_value_train:.2f}')

# Plot ROC curve for the test set
plt.plot(fpr_test, tpr_test, label=f'Test Set AUC = {auc_value_test:.2f}')

# Plot ROC curve for the validation set
# plt.plot(fpr_val, tpr_val, label=f'Validation Set AUC = {auc_value_val:.2f}', linestyle='-.')  # You can change the linestyle as you like

# Plot diagonal line
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')

# Labels, title, and other settings
plt.xlabel('1 - Specificity (False Positive Rate)', fontweight='bold')
plt.ylabel('Sensitivity (True Positive Rate)', fontweight='bold')
legend = plt.legend(loc='lower right')
plt.grid(False)
plt.show()

In [ ]:
#precision-recall
precision_train, recall_train, _ = precision_recall_curve(y_train, y_pred_proba_train,pos_label='tumor')
precision_test, recall_test, _ = precision_recall_curve(y_test, y_pred_proba_test,pos_label='tumor')
auprc_train = auc(recall_train, precision_train)
print("auprc_train:", auprc_train)
auprc_test = auc(recall_test, precision_test)
print("auprc_test:", auprc_test)

# precision_val, recall_val, _ = precision_recall_curve(y_val, y_pred_proba_val,pos_label='tumor')


plt.figure(figsize=(5, 5))
plt.plot(recall_train, precision_train, marker='.', label=f'Train Set AUCPRC = {auprc_train:.2f}', color='blue')

# Plot for test set
plt.plot(recall_test, precision_test, marker='.', label=f'Test Set AUCPRC = {auprc_test:.2f}', color='green')

# Plot for validation set
# plt.plot(recall_val, precision_val, marker='.', label='Validation', color='red')

# Labeling the axes and setting the title
plt.xlabel('Recall',fontweight='bold')
plt.ylabel('Precision',fontweight='bold')
#plt.title('Exosome Cohort benign vs cancer',fontweight='bold')

# Display the legend
plt.legend()
# Show the plot
plt.show()

In [ ]:
def compute_metrics_from_cm(cm):
    # Extracting values from the confusion matrix
    TP = cm[0, 0]
    FN = cm[0, 1]
    FP = cm[1, 0]
    TN = cm[1, 1]

    # Calculating Sensitivity and Specificity
    sensitivity = TP / (TP + FN)
    specificity = TN / (TN + FP)
    

    return sensitivity, specificity

In [ ]:
#p is the optimal value
y_pred_train_labels = ["tumor" if p >=  else "normal" for p in y_pred_proba_train]
y_pred_test_labels = ["tumor" if p >=  else "normal" for p in y_pred_proba_test]

In [ ]:
cm_test = confusion_matrix(y_test, y_pred_test_labels)
print("Confusion Matrix:")
print(cm_test)
sensitivity_test, specificity_test = compute_metrics_from_cm(cm_test)
print("sensitivity_test, specificity_test:")
print(sensitivity_test, specificity_test)

cm_train = confusion_matrix(y_train, y_pred_train_labels)
sensitivity_train, specificity_train = compute_metrics_from_cm(cm_train)
print("sensitivity_train, specificity_train:")
print(sensitivity_train, specificity_train)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools

def plot_confusion_matrix(y_true, y_pred, classes=None, figsize=(7, 5), text_size=14):
    cm = confusion_matrix(y_true, y_pred)
    cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    n_classes = cm.shape[0]

    if classes is None:
        classes = [str(i) for i in range(n_classes)]
        
    fig, ax = plt.subplots(figsize=figsize)
    cax = ax.matshow(cm_norm, cmap=plt.cm.Blues)
    fig.colorbar(cax)

    ax.set(title=' threhold', 
           xlabel='Predicted Label', 
           ylabel='True Label', 
           xticks=np.arange(n_classes), 
           yticks=np.arange(n_classes), 
           xticklabels=classes, 
           yticklabels=classes)
    ax.xaxis.set_label_position('bottom')
    ax.xaxis.tick_bottom()

    ax.yaxis.label.set_size(text_size)
    ax.xaxis.label.set_size(text_size)
    ax.title.set_size(text_size * 1.2)
    
    threshold = (cm_norm.max() + cm_norm.min()) / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, f'{cm[i, j]} ({cm_norm[i, j]*100:.1f}%)', 
                 horizontalalignment='center', 
                 color='white' if cm_norm[i, j] > threshold else 'black', 
                 size=text_size)
    


In [ ]:
classes = ['Normal', 'Tumor']
plot_confusion_matrix(y_test, y_pred_test_labels, classes)
plt.show()